# Modelo 03 – Preprocesado alternativo + SVM

En este notebook implementamos una aproximación diferente a la usada en el modelo 02.

Aquí aplicamos:
- Imputación **KNNImputer** para variables numéricas.
- Codificación **OneHotEncoder** para variables categóricas.
- Estandarización **StandardScaler**.
- Entrenamos un modelo **SVM (Support Vector Machine)** con kernel RBF.
- Generamos el archivo `submission.csv` para Kaggle.

Este notebook debe ser completamente reproducible y su salida debe coincidir con la enviada a Kaggle.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

## Cargar datos

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

## Separar variables predictoras (X) y objetivo (y)

In [3]:
X = train.drop("target", axis=1)
y = train["target"]

## Identificar columnas numéricas y categóricas

In [4]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

num_cols, cat_cols

## Pipeline de preprocesamiento (DIFERENTE al modelo 02)

En este modelo aplicamos:
- Imputación por **KNNImputer** en vez de SimpleImputer.
- Escalado con **StandardScaler**.
- Codificación con **OneHotEncoder**.

In [5]:
numeric_pipeline = Pipeline([
    ("imputer", KNNImputer(n_neighbors=5)),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
])

## Entrenar modelo SVM

In [6]:
svm_model = Pipeline([
    ("preprocess", preprocess),
    ("svm", SVC(kernel="rbf", C=2.0, gamma="scale", probability=True))
])

svm_model.fit(X, y)

## Generar predicciones al conjunto de prueba

In [7]:
pred = svm_model.predict(test)
pred[:10]

## Crear archivo submission.csv

In [8]:
submission = pd.DataFrame({
    "id": test["id"],
    "target": pred
})

submission.to_csv("submission.csv", index=False)
submission.head()

# Conclusión

Este modelo utiliza una estrategia alternativa al notebook 02 aplicando KNNImputer, OneHotEncoder y SVM.
Se generó el archivo `submission.csv`, listo para enviarse a Kaggle y evaluar su rendimiento.